In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Bibliotecas

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier

## 1. Carregamento dos datasets

In [1]:
# Carregando dados e verificando a quantidade de observações
df_train = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/train.csv')
df_test = pd.read_csv('/kaggle/input/costa-rican-household-poverty-prediction/test.csv')

df_all = df_train.append(df_test)

df_train.shape, df_test.shape, df_all.shape

In [1]:
# verificando os tipos de variaveis
df_all.info(max_cols=143)

## 1.1 Verificando duplicidades

In [1]:
df_all.duplicated().value_counts()

## 1.2 Verificando Missing Values

In [1]:
## verificando a quantidade de nulos
df_all.isna().sum().sort_values(ascending = False)[df_all.isnull().sum() >= 1]

#### v2a1 = valor do aluguel mensal <br/>v18q1 = quantidade de tablets<br/>rez_esc = anos atrasado na escola<br/> meaneduc = media dos anos de educação dos adultos <br/> SQBmeaned = raiz quadrado da média dos anos de educação dos adultos

## 1.3 Tratando Missing Values

### v18q1

In [1]:
# Análise dos tablets v18q
df_all['v18q'].value_counts()

In [1]:
# Quantidade de nulos bate com a quantidade de individuos que não possuem tablets
df_all['v18q1'].isna().sum()

In [1]:
# Prenchendo com 0 os valores nulos de v18q1
df_all['v18q1'].fillna(0, inplace=True)

A quantidade de nulos na variável v18q1 possuem a mesma quantidade de observações da variavel v18q (0). Subentende que os nulos são indivíduos que não possuem tablets. Logo aplicado tratativa para preencher com 0.

### v2a1

In [1]:
# Verificando os valores de aluguel (v2a1) para os chefes/as de familia (parentesco1 = 1)
df_all[df_all['parentesco1'] == 1]['v2a1'].isnull().sum()

In [1]:
# Prenchendo com -1 os valores nulos de v2a1 (tratativa do outlier)
df_all['v2a1'].fillna(-1, inplace=True)

Por se tratar de valores de pagamentos, o preenchimento com zero não é viável por se tratar de um valor que pode ser utilizado. Sendo assim, foi optado em identificar os nulos com -1 para serem detectados como outliers

### meaneduc

In [1]:
# Verificando a distancia entre a media e a mediana
df_all.meaneduc.mean(), df_all.meaneduc.median()

In [1]:
# Preenchendo os valores nulos de meaneduc com o valor de 9 anos de estudos (entre média e mediana), ou seja, uma tendência geral dos dados
df_all['meaneduc'].fillna(df_all['meaneduc'].mean(), inplace=True)

Variável com poucos nulos. Como a média é próximo da mediana, foi optado por utilizar a média como método de tratamento

### SQBmeaned

In [1]:
# Prenchendo com -1 os valores nulos de SQBmeaned (estratégia do outlier)
df_all['SQBmeaned'].fillna(-1, inplace=True)

Variável com poucos nulos. Não faz sentido o preenchimento com a média dos quadrados. Optado em identificar os nulos como outliers

### rez_esc

In [1]:
# Prenchendo com -1 os valores nulos de rez_esc (estratégia do outlier)
df_all['rez_esc'].fillna(-1, inplace=True)

Como o zero é uma variável possível de acontecer (0 anos de atrasos), foi optado em identificar os nulos como outliers

### Target

In [1]:
df_all['Target'].isna().value_counts()

Os valores batem com a divisão de base treino e teste. Sendo assim não será realizado tratativa de missing values pois será utilizado a mesma divisão original e não haverá missings.

### Verificando resultado do tratamento dos missing values

In [1]:
## verificando a quantidade de nulos
df_all.isna().sum().sort_values(ascending = False)[df_all.isnull().sum() >= 1]

## 1.4 Utilizando dicionário de dados

In [1]:
# verificando possiveis variaveis categoricas
df_all.select_dtypes('object').head()

#### Dicionario de dados
#### id - Identificador de um membro da família <br/> idhogar - Identificador de familia <br/> dependency - Taxa de dependência<br/> edjefa - anos de escolaridade do chefe de família do sexo feminino <br/> edjefe - anos de escolaridade do chefe de família do sexo masculino

In [1]:
# Tratando valores quantitativos de acordo com o dicionário 
mapeamento = {'yes': 1, 'no': 0}

df_all['edjefa'] = df_all['edjefa'].replace(mapeamento).astype(int)
df_all['edjefe'] = df_all['edjefe'].replace(mapeamento).astype(int)
df_all['dependency'] = df_all['dependency'].replace(mapeamento).astype(float)

In [1]:
# verificando possiveis variaveis categoricas
df_all.select_dtypes('object').head()

Analisando dataframe de treino

## 2. Separando dataframes

In [1]:
# Separando as colunas para treinamento
feats = [c for c in df_all.columns if c not in ['Id', 'idhogar', 'Target']]

In [1]:
# Separar os dataframes
train, test = df_all[~df_all['Target'].isnull()], df_all[df_all['Target'].isnull()]

Xtrain, Ytrain = train[feats], train[['Target']]
Xtest, Ytest = test[feats], test[['Target']]

Xtrain.shape, Ytrain.shape, Xtest.shape, Ytest.shape

## 3. Análise Exploratória

### Composição da variável Target

In [1]:
# Analisando variável Target
ax = sns.countplot(data=Ytrain, x="Target")
plt.show()

Nota-se enviesamento dos dados da variável Target

In [1]:
# Verificando valores absolutos
Ytrain['Target'].value_counts()

### Composição do grupo familiar

In [1]:
# Parentesco dos ocupantes das residências
var = 'parentesco1','parentesco2','parentesco3','parentesco4','parentesco5','parentesco6','parentesco7','parentesco8','parentesco9','parentesco10','parentesco11','parentesco12'
leg = ['Chefe de família','Cônjuge','Filho(a)','Enteado(a)','Genro/Nora','Neto(a)','Pai/Mãe','Sogro(a)','Irmão/Irmã','Cunhado(a)','Outro familiar','Outro não familiar']

y = []
for v in var: 
    y.append(Xtrain[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Parentesco")
plt.show()

### Perfil do chefe de família

In [1]:
# Sexo dos ocupantes das residências
var = 'male','female'
leg = ['Masculino','Feminino']

y = []
for v in var: 
    y.append(Xtrain[Xtrain['parentesco1'] == 1][v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Sexo")
plt.show()

### Tipo de Moradia

In [1]:
# Tipo de moradia
var = 'tipovivi1','tipovivi2','tipovivi3','tipovivi4','tipovivi5'
leg = ['Própria','Financiada','Alugada','Precária','Outras']

y = []
for v in var: 
    y.append(Xtrain[v].value_counts()[1])
    
plt.figure(figsize=(18,5))
sns.barplot(x=leg, y=y).set_title("Tipo de Moradia")
plt.show()

## 4. Balanceamento com Over-Sampling

In [1]:
# Verificando a distribuição das classes na variável Target
Ytrain["Target"].value_counts(normalize = True)

In [1]:
# Fazendo o over-sampling
ros = RandomOverSampler(random_state=42)
X_over,y_over= ros.fit_resample(Xtrain,Ytrain)

# Verificando os dados
y_over['Target'].value_counts(normalize = True)

## 5. Treinando o modelo

In [1]:
rf = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf.fit(X_over,y_over)

In [1]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [1]:
# submitendo
test[['Id', 'Target']].to_csv('submission.csv', index=False)

## 6. Utilizando balanceamento Under-sampling

In [1]:
# Fazendo o over-sampling
ros = RandomUnderSampler(random_state=42)
X_under,y_under= ros.fit_resample(Xtrain,Ytrain)

# Verificando os dados
y_under['Target'].value_counts(normalize = True)

In [1]:
rf = RandomForestClassifier(n_jobs=1, n_estimators=200, random_state=42)
rf.fit(X_under,y_under)

In [1]:
# Prever o Target de teste usando o modelo treinado
test['Target'] = rf.predict(test[feats]).astype(int)
test['Target'].value_counts(normalize=True)

In [1]:
# submitendo
test[['Id', 'Target']].to_csv('submission.csv', index=False)

## 7. Verificando a importância de cada variável

In [1]:
# Verificando o peso de cada coluna na predição
fig = plt.figure(figsize=(15, 20))
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()